# 1- Check features after reducing cardinality
(the tables used in this notebook are created by `scripts/model1_preprocess_3`)

In [1]:
import pandas as pd
import numpy as np

In [2]:
techniques_df = pd.read_pickle ('../scripts/tmp_m1pp_technique.pkl')
groups_df = pd.read_pickle ('../scripts/tmp_m1pp_group.pkl')

techniques_org_df = pd.read_pickle ('../scripts/tmp_m1pp_technique_org.pkl')
groups_org_df = pd.read_pickle ('../scripts/tmp_m1pp_group_org.pkl')

In [3]:
print (techniques_df['input_technique_software_id'].explode().value_counts())
print (groups_df['input_group_software_id'].explode().value_counts())
print (groups_org_df['input_group_software_id'].explode().value_counts())

input_technique_software_id
other    293
         201
s0154     72
s0363     72
s0260     72
        ... 
s0249     14
s0487     14
s0136     14
s0517     14
s0579     14
Name: count, Length: 272, dtype: int64
input_group_software_id
other    102
s0002     41
s0029     29
s0039     27
s0154     21
        ... 
s0070      2
s0607      2
s0203      2
s0105      2
s0662      2
Name: count, Length: 66, dtype: int64
input_group_software_id
s0002    41
s0029    29
s0039    27
s0154    21
s0363    14
         ..
s0004     1
s0056     1
s0579     1
s0436     1
s0275     1
Name: count, Length: 464, dtype: int64


In [4]:
's0105' in techniques_df['input_technique_software_id'].explode().unique()

False

In [5]:
techniques_df.columns

Index(['technique_ID', 'input_technique_data_sources',
       'input_technique_detection_name', 'input_technique_mitigation_id',
       'input_technique_platforms', 'input_technique_software_id',
       'input_technique_tactics', 'input_technique_description',
       'input_technique_interaction_rate'],
      dtype='object')

In [6]:
techniques_org_df['input_technique_software_id'][0]

['']

In [7]:
from statistics import mode
def get_feature_length_stats(df: pd.DataFrame()):
    for col in list (df.columns)[1:]:
        if not isinstance(df[col].iloc[0], list): print ('{col}: scalar'.format (col = col))
        else:
                list_lengths = df[col].apply(lambda x: len(x))
                avg_len = list_lengths.mean()
                mode_len = mode(list_lengths)
                max_len = list_lengths.max()
                print ('{col} \tmean_len: {mean_len}\tmode_len: {mode_len}\tmax_len: {max_len}'.format(col = col, mean_len= avg_len, mode_len = mode_len, max_len = max_len))
        
def get_vocab_size (df: pd.DataFrame):
    for feature_name in df.columns[1:]:
        if feature_name in ['input_group_description', 'input_technique_description', 'input_group_interaction_rate', 'input_technique_interaction_rate']: continue
        vocab_size = len(df[feature_name].explode().unique())
        print ('{feature_name}: vocab size = {vocab_size}'.format (feature_name = feature_name, vocab_size = vocab_size))

In [8]:
print ("BEFORE")
get_feature_length_stats (techniques_org_df)
get_feature_length_stats (groups_org_df)
print ("AFTER")
get_feature_length_stats (techniques_df)
get_feature_length_stats (groups_df)

BEFORE
input_technique_data_sources 	mean_len: 3.154859967051071	mode_len: 3	max_len: 14
input_technique_detection_name 	mean_len: 3.154859967051071	mode_len: 3	max_len: 14
input_technique_mitigation_id 	mean_len: 2.1301482701812193	mode_len: 1	max_len: 11
input_technique_platforms 	mean_len: 2.510708401976936	mode_len: 1	max_len: 10
input_technique_software_id 	mean_len: 14.179571663920923	mode_len: 1	max_len: 334
input_technique_tactics 	mean_len: 1.2932454695222406	mode_len: 1	max_len: 4
input_technique_description: scalar
input_group_software_id 	mean_len: 6.1911764705882355	mode_len: 1	max_len: 46
input_group_description: scalar
AFTER
input_technique_data_sources 	mean_len: 2.7528830313014825	mode_len: 1	max_len: 9
input_technique_detection_name 	mean_len: 2.7528830313014825	mode_len: 1	max_len: 9
input_technique_mitigation_id 	mean_len: 1.9588138385502472	mode_len: 1	max_len: 8
input_technique_platforms 	mean_len: 2.510708401976936	mode_len: 1	max_len: 10
input_technique_software

In [9]:
print ("BEFORE")
get_vocab_size (groups_org_df)
get_vocab_size (techniques_org_df)
print ("AFTER")
get_vocab_size (groups_df)
get_vocab_size (techniques_df)

BEFORE
input_group_software_id: vocab size = 464
input_technique_data_sources: vocab size = 106
input_technique_detection_name: vocab size = 106
input_technique_mitigation_id: vocab size = 44
input_technique_platforms: vocab size = 11
input_technique_software_id: vocab size = 636
input_technique_tactics: vocab size = 14
AFTER
input_group_software_id: vocab size = 66
input_group_tactics: vocab size = 15
input_technique_data_sources: vocab size = 15
input_technique_detection_name: vocab size = 15
input_technique_mitigation_id: vocab size = 17
input_technique_platforms: vocab size = 11
input_technique_software_id: vocab size = 272
input_technique_tactics: vocab size = 14


In [10]:
feature_name = 'input_technique_software_id'
print ('BEFORE')
print (techniques_org_df[feature_name].explode(feature_name).value_counts())
print ('AFTER')
print (techniques_df[feature_name].explode(feature_name).value_counts())

BEFORE
input_technique_software_id
         201
s0363     72
s0260     72
s0154     72
s0650     71
        ... 
s0112      1
s0001      1
s0016      1
s0014      1
s0026      1
Name: count, Length: 636, dtype: int64
AFTER
input_technique_software_id
other    293
         201
s0154     72
s0363     72
s0260     72
        ... 
s0249     14
s0487     14
s0136     14
s0517     14
s0579     14
Name: count, Length: 272, dtype: int64


---

In [11]:
for col in techniques_df.columns[1:-1]:
    print ('--------',col)
    feature_name = col
    print (techniques_df.explode(feature_name)[feature_name].value_counts())

-------- input_technique_data_sources
input_technique_data_sources
other                                                 312
command_command_execution                             268
process_process_creation                              226
network_traffic_network_traffic_content               108
file_file_modification                                104
process_os_api_execution                               94
file_file_creation                                     91
network_traffic_network_traffic_flow                   86
application_log_application_log_content                72
windows_registry_windows_registry_key_modification     65
network_traffic_network_connection_creation            61
module_module_load                                     54
file_file_access                                       47
file_file_metadata                                     44
                                                       39
Name: count, dtype: int64
-------- input_technique_detection_na

# 2 Check labels

In [12]:
labels_org_df = pd.read_pickle ('../scripts/tmp_m1_pp_label_org.pkl')
labels_df = pd.read_pickle ('../scripts/tmp_m1pp_label.pkl')

In [13]:
print (labels_org_df['label'].value_counts())
labels_org_df

label
0.0    49716
1.0     3052
Name: count, dtype: int64


,group_ID,technique_ID,label
0,G0099,T1548.002,0.0
1,G0099,T1134,0.0
2,G0099,T1134.002,0.0
3,G0099,T1134.001,0.0
4,G0099,T1531,0.0
...,...,...,...
52763,G0045,T1102.002,0.0
52764,G0045,T1102.001,0.0
52765,G0045,T1102.003,0.0
52766,G0045,T1047,1.0


In [14]:
print (labels_df['label'].value_counts())
labels_df

label
0.0    23113
1.0     2571
Name: count, dtype: int64


,group_ID,technique_ID,label
388,G0006,T1548.002,0.0
389,G0006,T1134,0.0
390,G0006,T1134.002,0.0
391,G0006,T1134.001,0.0
392,G0006,T1531,0.0
...,...,...,...
52763,G0045,T1102.002,0.0
52764,G0045,T1102.001,0.0
52765,G0045,T1102.003,0.0
52766,G0045,T1047,1.0


In [15]:
labels_org_df.columns == labels_df.columns

array([ True,  True,  True])

## 2b- Check resampled data

In [16]:
train_y_df = pd.read_csv ('../data/processed/model1/train_y_split.csv')
# train_cv_y_df = pd.read_csv ('../data/processed/model1/train_cv_y_split.csv')
cv_y_df = pd.read_csv ('../data/processed/model1/cv_y_split.csv')
# test_y_df = pd.read_csv ('../data/processed/model1/test_y_split.csv')
train_y_resampled_df = pd.read_csv ('../data/processed/model1/train_y_resampled.csv')


In [17]:
for df in [train_y_df, train_y_resampled_df,cv_y_df]:
    print (df.shape, 'no. of groups: {n}'.format(n =  df['group_ID'].nunique()))
    print (df['label'].value_counts())

(20480, 3) no. of groups: 54
label
0.0    18473
1.0     2007
Name: count, dtype: int64
(16007, 3) no. of groups: 54
label
0.0    12313
1.0     3694
Name: count, dtype: int64
(5204, 3) no. of groups: 14
label
0.0    4640
1.0     564
Name: count, dtype: int64


# 3 Check exported dataset

In [18]:
import tensorflow as tf

In [19]:
train_dataset = tf.data.Dataset.load ('../data/processed/model1/train_dataset/')
cv_dataset = tf.data.Dataset.load ('../data/processed/model1/cv_dataset/')
# test_dataset = tf.data.Dataset.load ('../data/processed/model1/test_dataset/')
inputs = cv_dataset.element_spec[0]

In [20]:
cv_dataset.element_spec[0]

{'input_technique_interaction_rate': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
 'input_technique_software_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_technique_platforms': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_tactics': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_technique_description': TensorSpec(shape=(768,), dtype=tf.float32, name=None),
 'input_technique_data_sources': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_interaction_rate': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
 'input_technique_tactics': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_software_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_description': TensorSpec(shape=(768,), dtype=tf.float32, name=None),
 'input_technique_mitigation_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'in

In [21]:
input_technique = [item for item in list(inputs.keys()) if item.startswith ('input_technique')]
input_technique

['input_technique_interaction_rate',
 'input_technique_software_id',
 'input_technique_platforms',
 'input_technique_description',
 'input_technique_data_sources',
 'input_technique_tactics',
 'input_technique_mitigation_id',
 'input_technique_detection_name']

In [22]:
input_group = [item for item in list(inputs.keys()) if item.startswith ('input_group')]
input_group

['input_group_tactics',
 'input_group_interaction_rate',
 'input_group_software_id',
 'input_group_description']

In [23]:
# import tensorflow as tf
# count_zeros = 0
# count_ones = 0
# for features, labels in train_dataset:
#     # 'labels' will be the second element in each tuple
#     labels_array = labels.numpy()  # Convert labels tensor to numpy array
    
#     # Count occurrences of 0 and 1 in the labels
#     count_zeros += (labels_array == 0).sum()
#     count_ones += (labels_array == 1).sum()

# # Print the counts
# print(f"Number of zeros: {count_zeros}")
# print(f"Number of ones: {count_ones}")

In [24]:
# count_zeros = 0
# count_ones = 0
# for features, labels in cv_dataset:
#     # 'labels' will be the second element in each tuple
#     labels_array = labels.numpy()  # Convert labels tensor to numpy array
    
#     # Count occurrences of 0 and 1 in the labels
#     count_zeros += (labels_array == 0).sum()
#     count_ones += (labels_array == 1).sum()

# # Print the counts
# print(f"Number of zeros: {count_zeros}")
# print(f"Number of ones: {count_ones}")